### Document Reranking

**Reranking** is the process of reordering a set of retrieved documents so that the most relevant ones appear first.  

**How it works:**
1. Retrieve candidate documents using a search method (e.g., BM25, vector search).
2. Use a reranker model (like **CohereRerank**) to score each document based on query relevance.
3. Sort the documents by their scores and select the top-k for use in applications like LLM-based Q&A.

**Benefits:**  
- Improves relevance of results
- Reduces noise from less relevant documents
- Enhances final answer quality when used with LLMs

### Reading and Storing Data in vector database

In [1]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [2]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",
)

In [3]:
import chromadb

client = chromadb.Client()
client = chromadb.PersistentClient(path="./chroma_langchain_db")

In [4]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
    id=1,
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
    id=2,
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
    id=3,
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
    id=4,
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
    id=5,
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
    id=6,
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
    id=7,
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
    id=8,
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
    id=9,
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
    id=10,
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['6b9418a1-54d6-4756-bc02-1eb5f6416a6d',
 '56b8419a-8f9a-47d0-820c-87a9d8f70638',
 'b6e82d1b-6e16-4a29-a2ca-00e79af93b3b',
 '1768ad85-f584-41c2-bafe-e4fb9a10d1ac',
 'd83e08a4-2af8-4dbd-86d3-7d5eea157426',
 '71b0f312-ce59-4655-900b-51fb1ae9e8ca',
 '3d920c1d-16d6-4b82-8095-6be04ee752f2',
 'a650981c-e064-40a5-8541-3fbb7c20293c',
 '729b748d-c799-42c6-af6c-77da559f2ec9',
 '5269e6ce-906c-4e91-8f48-33d87b14898a']

In [5]:
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,

)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]
* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]


In [8]:
retriever = vector_store.as_retriever(
    search_type="similarity",   # or "mmr", "similarity_score_threshold"
    search_kwargs={"k": 4}      # number of docs to retrieve
)

retrever_docs = retriever.invoke("LangChain provides abstractions to make working with LLMs easy")
print("retrever_docs",retrever_docs)

retrever_docs [Document(id='b6e82d1b-6e16-4a29-a2ca-00e79af93b3b', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'), Document(id='a650981c-e064-40a5-8541-3fbb7c20293c', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'), Document(id='d83e08a4-2af8-4dbd-86d3-7d5eea157426', metadata={'source': 'tweet'}, page_content="Wow! That was an amazing movie. I can't wait to see it again."), Document(id='6b9418a1-54d6-4756-bc02-1eb5f6416a6d', metadata={'source': 'tweet'}, page_content='I had chocolate chip pancakes and scrambled eggs for breakfast this morning.')]


In [ ]:
# COHERE_API_KEY = 

### Reranking with Cohererank

We use **CohereRerank** to reorder retrieved documents by relevance to the query. 

In [ ]:
import os
os.environ["CO_API_KEY"] = ""  # replace with your key

from langchain_cohere import CohereRerank
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever

compressor = CohereRerank(
    model="rerank-english-v3.0"   # Required
)

compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    "LangChain provides abstractions to make working with LLMs easy"
)

for i, doc in enumerate(compressed_docs, start=1):
    print(f"{i}. {doc.page_content} | metadata: {doc.metadata}")


1. Building an exciting new project with LangChain - come check it out! | metadata: {'source': 'tweet', 'relevance_score': 0.00095101306}
2. LangGraph is the best framework for building stateful, agentic applications! | metadata: {'source': 'tweet', 'relevance_score': 2.8409744e-05}
3. I had chocolate chip pancakes and scrambled eggs for breakfast this morning. | metadata: {'source': 'tweet', 'relevance_score': 5.1630764e-07}
